In [19]:
import sys
import numpy as np
import random

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import HoverTool

# parameters
N = 10 # N circles
num_filter = 145 # number of filters
dims = [100,100] # input_img size

# generate blobs
blob_size = 25
# top left corner coordinate
cX = 40
cY = 40
height = blob_size
width = height # square blobs

# input_img
input_img = np.ones(dims[0]*dims[1])
for i in range(cX, cX+width):
    for j in range(cY, cY+height):
        input_img[i*dims[1]+j] = 255

input_img = np.reshape(input_img,(100,100))

# attention window
gx = random.randint(2,98)
gy = random.randint(2,98)

pdelta = np.logspace(0, N-4, N-3, base=1.145)
tdelta = pow(9,pdelta) 
tdelta = np.concatenate(((0,3,6),tdelta),axis=0) # tdelta[9]=141.34, 100*sqrt(2)=141.42

delta=np.zeros(N)
for i in range(1,N):
    delta[i] = tdelta[i]-tdelta[i-1]
delta[0] = delta[1]
sigma2 = delta*delta/4 # sigma=delta/2
    
mu = np.zeros([num_filter, 2])
mu[0,0] = 0
mu[0,1] = 0

for i in range(1,7):
    mu[i,0] = np.cos(np.pi*((i-1)/3))*sum(delta[1:2])
    mu[i,1] = np.sin(np.pi*((i-1)/3))*sum(delta[1:2])
    
for i in range(7,19):
    mu[i,0] = np.cos(np.pi*((i-7)/6))*sum(delta[1:3])
    mu[i,1] = np.sin(np.pi*((i-7)/6))*sum(delta[1:3])
    
for i in range(3, N):
    for j in range(1,18+1):
        mu[18*(i-2)+j,0] = np.cos(np.pi*((j-1)/9))*sum(delta[1:i+1])
        mu[18*(i-2)+j,1] = np.sin(np.pi*((j-1)/9))*sum(delta[1:i+1])
    
mu_x = gx + mu[:,0]
mu_y = gy + mu[:,1]
    
a = np.reshape([np.arange(dims[0])]*num_filter, (num_filter,-1))
b = np.reshape([np.arange(dims[1])]*num_filter, (num_filter,-1))

mu_x = np.reshape(mu_x, (-1,1))
mu_y = np.reshape(mu_y, (-1,1))
mu_x_list = np.reshape(mu_x, [-1]) 
mu_y_list = np.reshape(mu_y, [-1]) 

Fx = -np.square(a - mu_x)
Fy = -np.square(b - mu_y)
Fx[0] = np.exp(Fx[0]/(2*sigma2[0]))
Fy[0] = np.exp(Fy[0]/(2*sigma2[0]))

for i in range(1,7):
    Fx[i] = np.exp(Fx[i]/(2*sigma2[1]))
    Fy[i] = np.exp(Fy[i]/(2*sigma2[1]))
    
for i in range(7,19):
    Fx[i] = np.exp(Fx[i]/(2*sigma2[2]))
    Fy[i] = np.exp(Fy[i]/(2*sigma2[2]))
    
for i in range(3,N):
    for j in range(1,18+1):
        Fx[18*(i-2)+j] = np.exp(Fx[18*(i-2)+j]/(2*sigma2[i])) # num_filter x dims[0]
        Fy[18*(i-2)+j] = np.exp(Fy[18*(i-2)+j]/(2*sigma2[i])) # num_filter x dims[1]
        
# normalize
Fx=Fx/np.reshape(np.sum(Fx,axis=1),(num_filter,1))
Fy=Fy/np.reshape(np.sum(Fy,axis=1),(num_filter,1))

#filter_img
filter_img=np.zeros(num_filter)
for i in range(num_filter):
    if ((mu_x[i]>=100) or (mu_x[i]<=0) or (mu_y[i]>=100) or (mu_y[i]<=0)):
        filter_img[i] = 1
    else:
        filter_img[i] = np.matmul(np.reshape(Fy[i], (1,1,-1)), np.matmul(input_img, np.reshape(Fx[i], (1,-1,1))))
         
hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 15px; font-weight: bold;">@desc</span>
            </div>
            <div>
                <span style="font-size: 12px;">Location</span>
                <span style="font-size: 12px; color: #320;">($x, $y)</span>
            </div>
        </div>
        """
    )


source = ColumnDataSource(
        data=dict(
            x=[gx],
            y=[gy],
            desc=['Att_window Center'],
        )
)

dot_source = ColumnDataSource(
        data=dict(
            mu_x=mu_x_list,
            mu_y=mu_y_list,
            #desc=['Att_window Center'],
        )
)

p = figure(title="Input Image (gx=%-d, gy=%-d)" %(gx,gy), x_range=(-100,200), y_range=(-100,200), tools=[hover])
iii = p.image(image=[input_img], x=0, y=0, dw=100, dh=100, palette="Greys256")
p.circle('mu_x', 'mu_y', size=3, source=dot_source, color="orange")
p.circle('x', 'y', size=5, source=source, color="red")

filter_source = ColumnDataSource(
        data=dict(
            x=[dims[0]/2],
            y=[dims[1]/2],
            desc=['Filterbank Center'],
        )
)


p_filter = figure(title="Filter Image", x_range=(-100, 200), y_range=(-100, 200))

p_filter.annular_wedge(x=dims[0]/2, y=dims[1]/2, inner_radius=0, outer_radius=sum(delta[0:1]),
                       start_angle=0, end_angle=np.pi*2, color=(filter_img[0],filter_img[0],filter_img[0]), line_color="yellow")

for i in range(1, 7):
    p_filter.annular_wedge(x=dims[0]/2, y=dims[1]/2, inner_radius=sum(delta[0:1]), outer_radius=sum(delta[0:2]),
                       start_angle=np.pi*((i-1)/3), end_angle=np.pi*(i/3), color=(filter_img[i],filter_img[i],filter_img[i]), line_color="yellow")
for i in range(7, 19):
    p_filter.annular_wedge(x=dims[0]/2, y=dims[1]/2, inner_radius=sum(delta[0:2]), outer_radius=sum(delta[0:3]),
                       start_angle=np.pi*((i-7)/6), end_angle=np.pi*((i-6)/6), color=(filter_img[i],filter_img[i],filter_img[i]), line_color="yellow")
for i in range(3, N):
    for j in range(1,18+1):
        p_filter.annular_wedge(x=dims[0]/2, y=dims[1]/2, inner_radius=sum(delta[0:i]), outer_radius=sum(delta[0:i+1]),
                       start_angle=np.pi*((j-1)/9), end_angle=np.pi*(j/9), color=(filter_img[18*(i-2)+j],filter_img[18*(i-2)+j],filter_img[18*(i-2)+j]), line_color="yellow")

p_filter.circle('x', 'y', size=8, source=filter_source, color="red")
pp = gridplot([[p, p_filter]], toolbar_location=None)
show(pp)


print("filter_img")

print("Circle 0: (delta=%f, sigma2=%f)"%(delta[0],sigma2[0]))
print("[0]: %f, " %filter_img[0])

print("Circle 1: (delta=%f, sigma2=%f)"%(delta[1],sigma2[1]))
for i in range(1, 7):
    print("[%d]: %f, " %(i,filter_img[i]), end='')
  
print("\n\nCircle 2: (delta=%f, sigma2=%f)"%(delta[2],sigma2[2]))
for i in range(7, 19):
    print("[%d]: %f, " %(i,filter_img[i]), end='')
    
for i in range(3, N):
    print("\n\nCircle %d: (delta=%f, sigma2=%f)"%(i,delta[i],sigma2[i]))
    for j in range(1,18+1):
        print("[%d]: %f, " %(18*(i-2)+j,filter_img[18*(i-2)+j]), end='')

filter_img
Circle 0: (delta=3.000000, sigma2=2.250000)
[0]: 1.001307, 
Circle 1: (delta=3.000000, sigma2=2.250000)
[1]: 1.000000, [2]: 1.000007, [3]: 1.087236, [4]: 3.214848, [5]: 1.087236, [6]: 1.000007, 

Circle 2: (delta=3.000000, sigma2=2.250000)
[7]: 1.000000, [8]: 1.000000, [9]: 1.000000, [10]: 1.001307, [11]: 3.214827, [12]: 48.707592, [13]: 94.222887, [14]: 48.707592, [15]: 3.214848, [16]: 1.001307, [17]: 1.000000, [18]: 1.000000, 

Circle 3: (delta=3.000000, sigma2=2.250000)
[19]: 1.000000, [20]: 1.000000, [21]: 1.000000, [22]: 1.000000, [23]: 1.000006, [24]: 1.098103, [25]: 22.991789, [26]: 154.827357, [27]: 231.684793, [28]: 243.642681, [29]: 231.684794, [30]: 154.835404, [31]: 23.121542, [32]: 1.101750, [33]: 1.000006, [34]: 1.000000, [35]: 1.000000, [36]: 1.000000, 

Circle 4: (delta=3.376780, sigma2=2.850661)
[37]: 1.000000, [38]: 1.000000, [39]: 1.000000, [40]: 1.000000, [41]: 1.000009, [42]: 1.384677, [43]: 74.606579, [44]: 241.659172, [45]: 254.738247, [46]: 254.947472